In [2]:
import spacy
# Need to run !python -m spacy download en_core_web_sm
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation

In [57]:
from pathlib import Path as pathl
import sys, os, re
from heapq import nlargest

In [4]:
#append relevant file paths
new_path = pathl('.')
parent = new_path.resolve().parent
sys.path.append(str(parent))
from pdf_parser import pipeline

In [5]:
data_path = parent / 'Data'
def get_text(filename):
    'Return text from a filename'
    pdf_file = data_path / filename
    text_dict = pipeline(filepath = str(pdf_file))
    text = list(text_dict.values())
    text = sum(text, [])
    text = [sentence.strip() for sentence in text]
    text = ' '.join(text)
    return text

In [6]:
nlp = spacy.load('en_core_web_sm')
punctuation = '!"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~\n'
def process_text(text):
    '''Remove stopwords, punctuation and add the remaining to a dictionary with num occurances as values. 
    Returns word frequencies'''
    doc = nlp(text)
    tokens = [token.text for token in doc]
    
    stopwords = list(STOP_WORDS)
    word_frequencies = {}
    for word in doc:
        if word.text.lower() not in stopwords:
            if word.text.lower() not in punctuation:
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1
                    
    return word_frequencies

In [60]:
#Check to see if last file path in sys is the root folder
current_dir = os.walk(sys.path[-1] + '/Data')
all_text = []
for file in next(current_dir)[-1]:
    try:
        text = get_text(file)
        all_text.append(process_text(text))
    except:
        pass

In [65]:
all_text[0]

{'t': 1,
 '+44': 1,
 '0': 1,
 '1455': 1,
 '558': 1,
 '600': 1,
 'Registered': 1,
 'En': 1,
 'gland': 1,
 '1932827': 1,
 'ered': 1,
 'Office': 2,
 'Hunter': 1,
 'Boulevard': 1,
 '   ': 1,
 'Magna': 1,
 'Park': 1,
 'Lutterworth': 1,
 'LE17': 1,
 '4XN': 1,
 'England': 1,
 'Modern': 4,
 'Slavery': 5,
 'Human': 3,
 'Trafficking': 2,
 'Statement': 1,
 '2019': 1,
 'Organisation': 1,
 'Structure': 1,
 'VWR': 11,
 'International': 6,
 'Ltd': 5,
 'employs': 1,
 '782': 1,
 'associates': 4,
 'UK': 2,
 'Situated': 1,
 'Leicestershire': 1,
 'Customer': 1,
 'Service': 1,
 'Centre': 1,
 'purpose': 1,
 '-built': 1,
 'warehouse': 1,
 '1985': 1,
 'continued': 1,
 'grow': 1,
 'welcomed': 1,
 'Jencons': 1,
 'Lab3': 1,
 'Basan': 1,
 'Peqlab': 1,
 'Hichrom': 1,
 'Therapak': 1,
 'meant': 1,
 'able': 2,
 'consistently': 1,
 'provide': 2,
 'unmatched': 1,
 'product': 1,
 'choice': 1,
 'customers': 1,
 'services': 1,
 'include': 1,
 'custom': 1,
 'manufacturing': 1,
 'support': 1,
 'research': 2,
 'laboratory': 